In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import random
import json
from pathlib import Path
import dotenv
import warnings

from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

from mp_api.client import MPRester
from pymatgen.core import Structure, Composition
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from robocrys.condense.mineral import MineralMatcher

plt.rcParams["font.size"] = 20
warnings.filterwarnings("ignore")

In [3]:
base_path = Path("./data/mp-40/")

### Set up API key

Save your materials project API key in a .env file in the same directory as this notebook. The file should look like this:

```
MP_API_KEY=your_api_key
```

In [4]:
dotenv.load_dotenv()
MP_API_KEY = os.getenv("MP_API_KEY")
SCOPUS_API_KEY = os.getenv("SCOPUS_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 1. Materials Project

### 1.1. Retrieving created_at from the Materials Project API

`created_at` is only available in the `mpr.materials.search`, so we will use this function to retrieve the registration date of the materials.

In [ ]:
with MPRester(MP_API_KEY) as mpr:
    total_docs = mpr.materials.search(
        num_sites=[0, 40],
        fields=[
            "material_id",
            "created_at",
        ],
    )

In [ ]:
data = []
for doc in total_docs:
    data.append(
        {
            "material_id": doc.material_id,
            "created_at": doc.created_at,
        }
    )
df_mp_created_at = pd.DataFrame(data)
# remove duplicates
df_mp_created_at = df_mp_created_at.drop_duplicates(subset=["material_id"])
# save to csv
df_mp_created_at.to_csv(base_path / "mp-created-at.csv", index=False)

### 1.2. Donwload snapshot with constraints of num_sites <= 40 and energy convex hull <= 0.25 eV and experimental = True

In [ ]:
with MPRester(MP_API_KEY) as mpr:
    docs = mpr.summary.search(
        num_sites=[0, 40],
        # energy_above_hull=[0, 0.25],
        theoretical=False,
        fields=[
            "material_id",
            "structure",
            "energy_above_hull",
            "band_gap",
            "theoretical",
        ],
    )

In [10]:
excluded_gas_list = [
    "H",
    "He",
    "N",
    "O",
    "F",
    "Ne",
    "Cl",
    "Ar",
    "Kr",
    "Xe",
    "Rn",
    "Fr",
    "Og",
]

In [ ]:
data = []
for doc in tqdm(docs):
    st = doc.structure
    elements = [elmt.symbol for elmt in st.composition.elements]

    if len(elements) == 1 and elements[0] in excluded_gas_list:
        print(elements)
        continue

    if max(st.lattice.abc) > 20:
        print(st.formula, st.lattice.abc)
        continue

    row = {
        "material_id": doc.material_id,
        "energy_above_hull": doc.energy_above_hull,
        "band_gap": doc.band_gap,
        "cif": st.to(fmt="cif"),
    }
    data.append(row)

df_mp_api = pd.DataFrame(data)
# remove duplicates
df_mp_api = df_mp_api.drop_duplicates(subset="material_id")
# shuffle
df_mp_api = df_mp_api.sample(frac=1, random_state=42).reset_index()
# save to csv
df_mp_api.to_csv(base_path / "mp-api.csv", index=False)

In [ ]:
plt.rcParams["font.size"] = 20
df_mp_api["energy_above_hull"].hist(bins=100)
plt.xlabel("Energy above hull (eV/atom)")
plt.ylabel("Number of materials")
plt.ylim(0, 1000)
# fill the region of interest with red (0 to 0.25 eV/atom)
plt.axvspan(0, 0.25, color="red", alpha=0.2)
plt.text(0.25, 800, "Meta-stable", color="red")

### 1.3. Download abstract using Crossref API

In [15]:
# refer to "./data/mp-40/get_abstract_from_materials_id.py"

In [ ]:
df_mp_api = pd.read_csv(base_path / "mp-api.csv")
material_ids = df_mp_api["material_id"].values
print(len(material_ids))

# Analyze the abstract data
print(f"Number of total materials: {len(material_ids)}")
path_abstract_data = base_path / "abstract_data"
abstract_data_files = list(path_abstract_data.glob("*.json"))
print(f"Number of abstract data: {len(abstract_data_files)}")

# make pandas dataframe
data = []
for file in abstract_data_files:
    d = json.load(open(file))
    material_id = file.stem.split("_")[0]
    d["material_id"] = material_id
    data.append(d)

df_abstract_data = pd.DataFrame(data)
# df_abstract_data.to_csv(base_path / "abstract-data.csv", index=False)

In [ ]:
# The number of reference papers per material
df_abstract_data["material_id"].value_counts().value_counts().sort_index().plot(
    kind="bar"
)
plt.xlabel("Number of references per material")
plt.ylabel("Number of materials")
plt.xlim(0, 10)

In [ ]:
# Histogram of publication year
df_year = df_abstract_data["year"].dropna()
df_year = df_year[df_year.str.isnumeric()]
df_year.astype(int).hist(bins=50)
plt.xlabel("Publication year")

In [13]:
# find NaN values
df = df_abstract_data[~df_abstract_data["abstract"].isna()]

In [ ]:
for i in range(1000):
    print(df.iloc[i]["abstract"], df.iloc[i]["url"])
    print("-" * 40)

### 1.4. Calculate parameters using pymatgen

In [ ]:
# calculate properties
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

mineral_matcher = MineralMatcher()


def calculate_property(data):
    st = Structure.from_str(data.cif, fmt="cif")
    sg = SpacegroupAnalyzer(st, symprec=0.1)
    data["composition"] = st.composition.reduced_composition.alphabetical_formula
    data["volume"] = st.volume
    data["density"] = st.density
    data["atomic_density"] = st.density
    data["crystal_system"] = sg.get_crystal_system()
    data["space_group_symbol"] = sg.get_space_group_symbol()
    data["space_group_number"] = sg.get_space_group_number()
    try:
        data["mineral_name"] = mineral_matcher.get_best_mineral_name()["type"]
    except:
        data["mineral_name"] = None
    return data


df_mp_api = pd.read_csv(base_path / "mp-api.csv")
df_mp_total = df_mp_api.parallel_apply(calculate_property, axis=1)
df_mp_total.to_csv(base_path / "mp-total.csv", index=False)

### 1.5. Make test set registered after 

In [5]:
df_mp_total = pd.read_csv(base_path / "mp-total.csv")
df_mp_total = df_mp_total.drop(columns=["index"])

In [112]:
# stable label
df_mp_total["stability"] = df_mp_total["energy_above_hull"].apply(
    lambda x: "stable" if x == 0 else "metastable" if x < 0.25 else "unstable"
)
# metallic label
df_mp_total["metallic"] = df_mp_total["band_gap"].apply(
    lambda x: "metallic" if x < 0.1 else "insulator"
)

In [ ]:
df_total = df_mp_total.sample(frac=1, random_state=42)
df_test = df_total.iloc[:1000]
df_train_val = df_total.iloc[1000:]
num_train = int(len(df_train_val) * 0.90)
df_train = df_train_val.iloc[:num_train]
df_val = df_train_val.iloc[num_train:]
print(len(df_train), len(df_val), len(df_test))

In [114]:
# save
df_train.to_csv(base_path / "train.csv", index=False)
df_val.to_csv(base_path / "val.csv", index=False)
df_test.to_csv(base_path / "test.csv", index=False)

# 2. Text Prompts (MatCap)

In [5]:
from openai import OpenAI

In [6]:
df_mp_total = pd.read_csv(base_path / "mp-total.csv")
df_abstract_data = pd.read_csv(base_path / "abstract-data.csv")
df_abstract_data = df_abstract_data[~df_abstract_data["abstract"].isna()]

In [38]:
def composition_augmentation(composition: Composition):
    comp_list = list(
        set(
            [
                composition.reduced_formula,  # reduced formula
                composition.reduced_composition.alphabetical_formula,  # reduced alphabetical formula
                composition.reduced_composition.iupac_formula,  # reduced IUPAC formula
                composition.reduced_composition.hill_formula,  # reduced Hill formula
            ]
        )
    )
    return comp_list

In [39]:
mineral_matcher = MineralMatcher()
for idx, row in tqdm(df_mp_total.iterrows(), total=len(df_mp_total)):
    st = Structure.from_str(row["cif"], fmt="cif")
    # composition
    composition = st.composition
    reduced_formula = composition.reduced_formula
    comp_list = composition_augmentation(composition)

    # crystal system
    crystal_system = row["crystal_system"]

    # stability: stable < 0 eV, metastable < 0.25 eV, unstable > 0.25 eV
    stability = (
        "stable"
        if row["energy_above_hull"] == 0
        else "metastable" if row["energy_above_hull"] < 0.25 else "unstable"
    )

    # metallic: metallic < 0.1 eV, insulator > 0.1 eV
    metallic = "metallic" if row["band_gap"] < 0.1 else "insulator"

    # mineral
    # mineral_name = "" if pd.isna(row["mineral_name"]) else row["mineral_name"]
    mineral_name = ""  # TODO !!!!! remove this line

    # paper data
    paper_data = ""
    if row["material_id"] in df_abstract_data["material_id"].values:
        df = df_abstract_data[df_abstract_data["material_id"] == row["material_id"]]
        for i, (_, row) in enumerate(df.iterrows()):
            paper_data += f"paper{i+1} - title: {row['title'][2:-2]} | abstract: {row['abstract']}\n"
    paper_data = paper_data[:3000]  # limit the length of paper_data

    template = f"""provide concise captions for "{reduced_formula}" with the following properties:


{crystal_system}
{stability}
{metallic}
{mineral_name}
{paper_data}

Here are some examples:
1. Orthorhombic crystal structure of ZnMnO4
2. metastable crystal structure of LiO2
3. Si1 C1 crystal structure with metallic properties

Please provide "five concise captions" to describe the crystal structure with the various compound name: {", ".join(comp_list)}
"""
    print(template)
    print("-" * 40)
    if idx == 1:
        break

Reading file /home/hyunsoo/anaconda3/envs/chemeleon/lib/python3.10/site-packages/robocrys/condense/mineral_db.json.gz: 0it [00:00, ?it/s]##4   | 116/180 [00:00<00:00, 1141.17it/s]
Decoding objects from /home/hyunsoo/anaconda3/envs/chemeleon/lib/python3.10/site-packages/robocrys/condense/mineral_db.json.gz: 100%|##########| 180/180 [00:00<00:00, 1313.10it/s]
  0%|          | 1/35318 [00:00<11:28, 51.33it/s]

provide concise captions for "MgZnAg2" with the following properties:


cubic
stable
metallic

paper1 - title: Deformation of single crystals of the L21 ordered Ag2MgZn | abstract: The orientation and temperature dependence of slip geometry and yield stress in single crystals of the L21 ordered Ag2MgZn has been studied in compression in the temperature range 290 to 580 K. The slip direction in Ag2MgZn is exclusively 〈1 1 1〉 in this temperature range, but the slip plane varies with crystal orientations; slip occurs on (-2 1 1) for orientations near the [0 1 1]-[-1 1 1] boundary, while for the other orientations in the [0 0 1]-[0 1 1]-[-1 1 1] unit triangle it occurs on the (-1 0 1). The critical resolved shear stress (CRSS) for slip on both the (-1 0 1) [1 1 1] and (-2 1 1) [1 1 1] systems increases abnormally with increasing temperature and reaches a maximum peak at about 0.92 of the critical temperature Tc, for the L21-type order. The peak temperature and the shape of the CRSS versus 

In [40]:
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [44]:
print(template)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": template,
        }
    ],
    model="gpt-4o-mini",
)

provide concise captions for "CdSbS2Br" with the following properties:


monoclinic
metastable
insulator

paper1 - title: Synthesis, Structure, and Properties of a New Family of Mixed‐Framework Chalcohalide Semiconductors: CdSbS<sub>2</sub>X (X: Cl, Br), CdBiS<sub>2</sub>X (X: Cl, Br), and CdBiSe<sub>2</sub>X (X: Br, I). | abstract: <jats:title>Abstract</jats:title><jats:p>ChemInform is a weekly Abstracting Service, delivering concise information at a glance that was extracted from about 200 leading journals. To access a ChemInform Abstract, please click on HTML or PDF.</jats:p>


Here are some examples:
1. Orthorhombic crystal structure of ZnMnO4
2. metastable crystal structure of LiO2
3. Si1 C1 crystal structure with metallic properties

Please provide "five concise captions" to describe the crystal structure with the various compound name: Br Cd S2 Sb, Cd1 Sb1 S2 Br1, CdSbS2Br, Br1 Cd1 S2 Sb1



In [ ]:
output = chat_completion.choices[0].message.content
print(output)

1. Monoclinic structure of CdSbS2Br, a metastable insulator.  
2. Br Cd S2 Sb exhibits a unique monoclinic crystal formation.  
3. Cd1 Sb1 S2 Br1: A metastable insulator with monoclinic geometry.  
4. CdSbS2Br demonstrates distinctive properties as a layered chalcohalide.  
5. Br1 Cd1 S2 Sb1 showcases a monoclinic structure with insulating characteristics.


In [ ]:
# update trian, test, val
for split in ["train", "val", "test"]:
    df = pd.read_csv(base_path / f"{split}.csv")
    df = pd.merge(df, df_prompts, on="material_id")
    df.to_csv(base_path / f"{split}.csv", index=False)